# Balance the workload between two groups of workers

This example uses the more general _minimum cost flow_ solver. In this example 

This requires different node numbering than the original assignment example, because the minimum cost solver requires all nodes to have distinct numbering.

In [1]:
from ortools.graph import pywrapgraph
import time

## Define the directed graph for the flow. 

These are divided into three sub-arrays:

1. Arcs leading out of the source
2. Flows between workers and tasks, for which there are non-zero costs
3. Arcs leading into the sink.

The data also includes the vector `supplies`, showing the supply at each node $0-9$.

The condition of _flow in equals flow out_ forces the flow out of each worker to be $1$. This also forces each task to be be assigned to a worker for this configuration.

The workers are denoted by nodes $1-6$, divided into Team A and Team B.
The tasks are nodes $7-10$.

Node 11 is connected to the nodes for Team A, and node 12 is connected to the nodes for Team B.

Each team can perform at most two of the tasks.

In [5]:
def main():
    """
    Solve an assignment problem with MinCostFlow
    """
    
    # instantiate a simple SimpleMinCostFlow solver
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()
    
    # define the directed graph for the flow
    team_A = [1, 2, 5]
    team_B = [2, 4, 6]
    
    start_nodes =  ([0, 0] + [11, 11, 11] + [12, 12, 12] + [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4,
                                                            5, 5, 5, 5, 6, 6, 6, 6] + [7, 8, 9, 10])
    end_nodes   =  ([11, 12] + team_A + team_B + [7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10, 7, 8, 9, 10,
                                                 7, 8, 9, 10, 7, 8, 9, 10] + [13, 13, 13, 13])
    capacities  =  ([2, 2] + [1, 1, 1] + [1, 1, 1] + [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
                                                     1, 1, 1, 1, 1, 1, 1, 1] + [1, 1, 1, 1])
    costs       = ([0, 0] + [0, 0, 0] + [0, 0, 0] +
                 [90, 76, 75, 70, 35, 85, 55, 65, 125, 95, 90, 105, 45, 110, 95, 115, 60, 105,
                  80, 75, 45, 65, 110, 95] + [0, 0, 0, 0])

    # define an array of supplies at each node
    supplies = [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4]
    source = 0
    sink = 13
    
    # add each arc
    for i in range(len(start_nodes)):
        min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i], capacities[i], costs[i])
    
    # add node supplies
    for i in range(len(supplies)):
        min_cost_flow.SetNodeSupply(i, supplies[i])
    
    # invoke the solver and display the solution
    if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
        print('Total cost = %d.' % (min_cost_flow.OptimalCost()))
        print()
        for arc in range(min_cost_flow.NumArcs()):
            
            # can ignore the nodes from source and to sink
            if min_cost_flow.Tail(arc) != source and min_cost_flow.Head(arc) != sink and min_cost_flow.Tail(arc) != 11 and min_cost_flow.Tail(arc) != 12:
                
                # arcs in the solution have a flow value of 1. Their start and
                # end nodes give an assignment of worker to task.
                if min_cost_flow.Flow(arc) > 0:
                    print('Worker %d assigned to task %d; cost = %d.' % (
                    min_cost_flow.Tail(arc),
                    min_cost_flow.Head(arc),
                    min_cost_flow.UnitCost(arc)))
                    
    else:
        print('There was an issue; no optimal solution found.')   

In [6]:
if __name__ == "__main__":
    start_time = time.clock()
    main()
    print()
    print("Time = ", time.clock() - start_time)

Total cost = 225.

Worker 1 assigned to task 10; cost = 70.
Worker 2 assigned to task 7; cost = 35.
Worker 2 assigned to task 9; cost = 55.
Worker 6 assigned to task 8; cost = 65.

Time =  0.0008890000000003617
